# Cleaning Data

In [17]:
import pandas as pd
import numpy as np

In [7]:
from multiprocessing import Pool, cpu_count

## 1. Labeling Based on Expiration Date

for this step, I first sort the raw data (transactions.csv) by the users' ID: 

In [1]:
#!cat ./Data/transactions.csv | sort --field-separator=',' --key=1 > sorted_trans.csv

Second step is to make labels based on expiration date. The code below, forst read all the id's as key and the info for each id as values (each key can have more than 1 value since the user can have many transactions through time.)
Each user considered as churned in 2 cases:
1. If the most recent transaction has been canceled; 
or
2. If the expiration date for the most recent transaction has been past. (the current date has been defined as the latest date available in transaction date (Among all the users).

In [ ]:
'''
#Python code for churn labeling (labeling.py):
#! /usr/bin/env python

from operator import itemgetter
import csv
import sys
from datetime import datetime


def compute_lf(subscription_map):
  date_of_latest_transaction = datetime.strptime('20170228','%Y%m%d').date()
  for k,records in subscription_map.iteritems():
    # a set of transactions are considered bad if a subscription is canceled and then
    # it's followed by more transactions. 
    bad_transactions = False
    for record in records[:-1]:
      if record[-1] == '1':
        bad_transactions = True
        break

    if bad_transactions:
      continue
      #print 'Found bad transactions for %s' %k
    # This is the date of the first transaction
    # This is the expiration date of the last transaction
    expiration_date = datetime.strptime(records[-1][7],'%Y%m%d').date()

    subscription_canceled = records[-1][-1] == '1'
    churn = '0'
    if subscription_canceled == False:
      # if the expiration date of the last transaction is before the date
      # of the latest transaction in the dataset, this is considered to be a churn.
      if expiration_date < date_of_latest_transaction:
        subscription_canceled = True
        #print 'This is actually a churn: %s' %k
    if subscription_canceled:
      churn = '1'
    print k +  ',' + churn


def generate_clean_data():
  input_file = open('./sorted_trans.csv', 'r')
  lines = input_file.readlines()
  lines = [line.strip() for line in lines]
  sorted_lines = sorted(lines, key= lambda line : line.split(',')[7])
  subscription_map = {}

  for line in sorted_lines:
    transaction_id = line.split(',')[0]
    transaction_record = line.split(',')[1:]
    if subscription_map.has_key(transaction_id):
      subscription_map[transaction_id].append(transaction_record)
    else:
       subscription_map[transaction_id] = [transaction_record]

  return subscription_map

if __name__ == '__main__':
  subscription_map = generate_clean_data()
  compute_lf(subscription_map)

'''

In [ ]:
#! python labeling.py > labeled_churn.csv &

Now that we have labeled data, we can split them to have training and test data set. Its better if we check and make sure that our labeling is consistent with the labeled data provided by KKBOX.

# Using Churn Dataset Provided by KKBOX

In [2]:
train = pd.read_csv('../Data/train.csv')
test = pd.read_csv('../Data/sample_submission_zero.csv')

transactions = pd.read_csv('../Data/transactions.csv', usecols=['msno'])
#transactions = pd.concat((transactions, pd.read_csv('transactions_v2.csv', usecols=['msno'])), axis=0, ignore_index=True).reset_index(drop=True)
transactions = pd.DataFrame(transactions['msno'].value_counts().reset_index())
transactions.columns = ['msno','trans_count']
train = pd.merge(train, transactions, how='left', on='msno')
test = pd.merge(test, transactions, how='left', on='msno')
transactions = []; print('transaction merge...')

#user_logs = pd.read_csv('../input/user_logs_v2.csv', usecols=['msno'])
user_logs = pd.read_csv('../Data/user_logs.csv', usecols=['msno'])
#user_logs = pd.concat((user_logs, pd.read_csv('../input/user_logs_v2.csv', usecols=['msno'])), axis=0, ignore_index=True).reset_index(drop=True)
user_logs = pd.DataFrame(user_logs['msno'].value_counts().reset_index())
user_logs.columns = ['msno','logs_count']
train = pd.merge(train, user_logs, how='left', on='msno')
test = pd.merge(test, user_logs, how='left', on='msno')
user_logs = []; print('user logs merge...')

members = pd.read_csv('../Data/members_v3.csv')
train = pd.merge(train, members, how='left', on='msno')
test = pd.merge(test, members, how='left', on='msno')
members = []; print('members merge...') 

transaction merge...
user logs merge...
members merge...


In [3]:
gender = {'male':1, 'female':2}
train['gender'] = train['gender'].map(gender)
test['gender'] = test['gender'].map(gender)

train = train.fillna(0)
test = test.fillna(0)

In [4]:
transactions = pd.read_csv('../Data/transactions.csv') #pd.read_csv('../input/transactions.csv')
#transactions = pd.concat((transactions, pd.read_csv('../input/transactions_v2.csv')), axis=0, ignore_index=True).reset_index(drop=True)
transactions = transactions.sort_values(by=['transaction_date'], ascending=[False]).reset_index(drop=True)
transactions = transactions.drop_duplicates(subset=['msno'], keep='first')

train = pd.merge(train, transactions, how='left', on='msno')
test = pd.merge(test, transactions, how='left', on='msno')
transactions=[]

In [ ]:

def transform_df(df):
    df = pd.DataFrame(df)
    df = df.sort_values(by=['date'], ascending=[False])
    df = df.reset_index(drop=True)
    df = df.drop_duplicates(subset=['msno'], keep='first')
    return df

def transform_df2(df):
    df = df.sort_values(by=['date'], ascending=[False])
    df = df.reset_index(drop=True)
    df = df.drop_duplicates(subset=['msno'], keep='first')
    return df

df_iter = pd.read_csv('../Data/user_logs.csv', low_memory=False, iterator=True, chunksize=10000000)
last_user_logs = []
i = 0 #~400 Million Records - starting at the end but remove locally if needed
for df in df_iter:
    if i>35:
        if len(df)>0:
            print(df.shape)
            p = Pool(cpu_count())
            df = p.map(transform_df, np.array_split(df, cpu_count()))   
            df = pd.concat(df, axis=0, ignore_index=True).reset_index(drop=True)
            df = transform_df2(df)
            p.close(); p.join()
            last_user_logs.append(df)
            print('...', df.shape)
            df = []
    i+=1


In [12]:
last_user_logs.append(transform_df(pd.read_csv('../Data/user_logs_v2.csv')))
last_user_logs = pd.concat(last_user_logs, axis=0, ignore_index=True).reset_index(drop=True)
last_user_logs = transform_df2(last_user_logs)

train = pd.merge(train, last_user_logs, how='left', on='msno')
test = pd.merge(test, last_user_logs, how='left', on='msno')
last_user_logs=[]

In [13]:
train = train.fillna(0)
test = test.fillna(0)

In [14]:
print type(test)

<class 'pandas.core.frame.DataFrame'>


In [15]:
test.to_csv("clean_test.csv")

In [16]:
train.to_csv("clean_train.csv")

lets check and see if we can only read 2 columns of logs:

In [1]:
#!head ../Data/user_logs.csv

msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,20150513,0,0,0,0,1,1,280.3350
rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,20150709,9,1,0,0,7,11,1658.9480
yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,20150105,3,3,0,0,68,36,17364.9560
yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,20150306,1,0,1,1,97,27,24667.3170
yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,20150501,3,0,0,0,38,38,9649.0290
yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,20150702,4,0,1,1,33,10,10021.5200
yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,20150830,3,1,0,0,4,7,1119.5550
yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,20151107,1,0,0,0,4,5,938.0220
yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,20160110,2,0,1,0,11,6,3004.0680


In [ ]:
##user_logs = pd.read_csv('../Data/user_logs.csv', usecols=['msno'])
#user_logs = pd.concat((user_logs, pd.read_csv('../input/user_logs_v2.csv', usecols=['msno'])), axis=0, ignore_index=True).reset_index(drop=True)
#user_logs = pd.DataFrame(user_logs['msno'].value_counts().reset_index())
#user_logs.columns = ['msno','logs_count']

In [1]:
'''
logs_count = pd.DataFrame(user_logs['msno'].value_counts().reset_index())
logs_count.columns = ['msno','logs_count']
user_logs_25 = user_logs.groupby('msno')['num_25'].statistics.median()
user_logs_25.columns = ['msno','med_25']
user_logs = []
'''

"\nlogs_count = pd.DataFrame(user_logs['msno'].value_counts().reset_index())\nlogs_count.columns = ['msno','logs_count']\nuser_logs_25 = user_logs.groupby('msno')['num_25'].statistics.median()\nuser_logs_25.columns = ['msno','med_25']\nuser_logs = []\n"